In [1]:
import pandas as pd

In [2]:
item_dt = pd.read_csv("items.csv", sep='|')
trans_dt = pd.read_csv("transactions.csv", sep='|')
eval_dt = pd.read_csv("evaluation.csv")

In [3]:
# Converting ID's to string
item_dt['itemID']=item_dt['itemID'].astype(str)
trans_dt['itemID']=trans_dt['itemID'].astype(str)
trans_dt['sessionID']=trans_dt['sessionID'].astype(str)
eval_dt['itemID']= eval_dt['itemID'].astype(str)

In [4]:
# Create column based on number of items in session
trans_dt['session_count'] = trans_dt.groupby('sessionID')['itemID'].transform('count')

#Filter by session_count>=2 to include sessions with more than 2 items.
sig_trans_dt = trans_dt[trans_dt['session_count']>=2]

In [5]:
# We can see only 1/3 of transaction data are remaining.
print(trans_dt.shape,sig_trans_dt.shape)

(365143, 6) (129501, 6)


In [6]:
# We don't need session_count from now
del trans_dt['session_count']
del sig_trans_dt['session_count']

In [7]:
sig_trans_dt.head()

,sessionID,itemID,click,basket,order
7,7,14576,1,1,0
8,7,17731,2,1,0
13,12,30277,1,0,0
14,12,29508,1,1,0
15,12,75659,1,0,0


In [8]:
#Only 304 items in evaluation dataset are included in sig_trans_dt
eval_dt[eval_dt['itemID'].isin(sig_trans_dt['itemID'])].shape

(304, 1)

In [9]:
eval_dt[eval_dt['itemID'].isin(sig_trans_dt['itemID'])]

,itemID
3,41371
8,56794
10,62060
16,24603
24,77956
...,...
983,56782
985,50648
990,58358
994,23570


In [10]:
#Let's work with one of the items. (41373, Dryadenhain & Dschinnenzauber (Märchenanthologie))
item_dt[item_dt['itemID'] == "41373"]

,itemID,title,author,publisher,main topic,subtopics
17261,41373,Werewolves of Rebellion - Im Bann der Wölfe,Ana Lee Kennedy,Bastei Lübbe AG,FMR,[FRT]


In [11]:
item_included_sessions = sig_trans_dt[sig_trans_dt['itemID'] == "41371"]['sessionID']

In [12]:
#Only one session includes item 41371
item_included_sessions

363704    278321
Name: sessionID, dtype: object

In [13]:
included_dt = sig_trans_dt[sig_trans_dt['sessionID'].isin(item_included_sessions)]

In [14]:
# This session also includes item 29539 and 54683.
included_dt

,sessionID,itemID,click,basket,order
363704,278321,41371,1,1,0
363705,278321,29539,0,1,0
363706,278321,54689,0,1,0


In [15]:
included_dt.groupby('itemID').sum()['click'].sort_values(ascending=False)

itemID
41371    1
29539    0
54689    0
Name: click, dtype: int64

In [17]:
# Let's see which items are they. We can find that these books have similar topics and same publisher.
item_dt[item_dt['itemID'].isin(included_dt['itemID'])]

,itemID,title,author,publisher,main topic,subtopics
5461,54689,Aventurischer Almanach Hardcover,"Florian Don-Schauen, Daniel Simon Richter",Ulisses Spiel & Medien,FM,[WDHW]
17169,29539,Die Siebenwindküste Albernia & der Windhag,Daniel Simon Richter,Ulisses Spiel & Medien,WDHW,"[FMB,WDHW]"
19660,41371,Dryadenhain & Dschinnenzauber (Märchenanthologie),"Philipp Busch, Jonathan Driedger, Lena Kalupne...",Ulisses Spiel & Medien,FM,"[FMB,WDHW]"


## Let's start with whole items which are included in evaluation data

In [18]:
eval_included_sessions = sig_trans_dt[sig_trans_dt['itemID'].isin(eval_dt['itemID'])]['sessionID']

In [19]:
eval_included_trans_dt = sig_trans_dt[sig_trans_dt['sessionID'].isin(eval_included_sessions)]

In [20]:
# We get dataframe with 15551 rows and 5 columns
eval_included_trans_dt

,sessionID,itemID,click,basket,order
190,148,48175,1,0,0
191,148,47684,1,0,0
251,205,15528,0,1,0
252,205,74398,1,0,0
253,205,2417,2,0,0
...,...,...,...,...,...
365118,279343,15609,1,0,0
365119,279343,33928,9,0,0
365120,279343,68184,1,0,0
365121,279343,67832,2,0,0


In [21]:
# Item which are included in sig_trans_dt and eval_dt
items = eval_dt[eval_dt['itemID'].isin(sig_trans_dt['itemID'])]['itemID']

In [22]:
# We will get our recommendations as dictionary
recommendations = {}

In [23]:
for i in items:
    item_included_sessions = eval_included_trans_dt[eval_included_trans_dt['itemID']==i]['sessionID'] # Sessions which items are in.
    included_dt = sig_trans_dt[sig_trans_dt['sessionID'].isin(item_included_sessions)] # Filtering dataframe by session
    # Instead of item_count, we can also use sum of other columns, such as orders or clicks.
    item_count = included_dt[included_dt['itemID']!= i].groupby('itemID').count()['sessionID'].sort_values(ascending=False)
    # item_order = included_dt[included_dt['itemID']!= i].groupby('itemID').sum()['order'].sort_values(ascending=False)
    recommended_items = item_count.head(5).index.to_list() # Get top 5 values.
    recommendations[i] = recommended_items

In [24]:
recommendations

{'41371': ['29539', '54689'],
 '56794': ['44420', '2121', '10305', '63769', '51931'],
 '62060': ['1136', '37310', '23422', '34366', '34906'],
 '24603': ['32145', '10522', '14442', '29782', '30561'],
 '77956': ['70706', '14442', '25462', '32948', '33848'],
 '67776': ['68187', '5558', '67039', '11918', '45814'],
 '53929': ['75204', '17783', '64654', '6618', '24285'],
 '2319': ['16123', '22061', '24844', '27457', '30023'],
 '62494': ['27493', '23319', '50539', '76022', '74871'],
 '21497': ['77708', '19761', '48669', '36098', '75034'],
 '53008': ['67796', '52217', '59779', '15175', '21438'],
 '39308': ['10158', '20641', '40692', '42058', '46617'],
 '4690': ['13869', '52146', '15840', '73855', '54699'],
 '65353': ['101', '63819', '48733', '4973', '49979'],
 '61593': ['40399', '61749', '68724', '8643', '66814'],
 '74398': ['75305', '15606', '19379', '37255', '72081'],
 '38760': ['29711', '32266', '45237', '5291'],
 '17665': ['50330', '38034', '37594', '9282', '37794'],
 '56282': ['9018', '25

In [25]:
recommendation_dt = pd.DataFrame()

In [26]:
recommendation_dt['itemID'] = recommendations.keys()
recommendation_dt = recommendation_dt.set_index('itemID')

In [27]:
recommendation_dt['recommendation_1'] = None
recommendation_dt['recommendation_2'] = None
recommendation_dt['recommendation_3'] = None
recommendation_dt['recommendation_4'] = None
recommendation_dt['recommendation_5'] = None

In [28]:
for i in recommendation_dt.index:
    for j in range(len(recommendations[i])):
        recommendation_dt.loc[i,"recommendation_%d"%(j+1)] = recommendations[i][j]    

In [29]:
recommendation_dt

,recommendation_1,recommendation_2,recommendation_3,recommendation_4,recommendation_5
itemID,,,,,
41371,29539,54689,None,None,None
56794,44420,2121,10305,63769,51931
62060,1136,37310,23422,34366,34906
24603,32145,10522,14442,29782,30561
77956,70706,14442,25462,32948,33848
...,...,...,...,...,...
56782,19466,70481,66218,10191,59231
50648,63736,None,None,None,None
58358,6095,26078,52394,57469,9046


In [35]:
recommendation_dt['recommendation_2'].isna().sum()/1000

0.026

In [36]:
recommendation_dt['recommendation_3'].isna().sum()/1000

0.045

In [37]:
recommendation_dt['recommendation_4'].isna().sum()/1000

0.059

In [39]:
(304-recommendation_dt['recommendation_5'].isna().sum())/1000

0.233

In [29]:
recommendation_dt.to_csv("trans_recommend.csv")

In [31]:
item_dt[item_dt['itemID'].isin(["56794","44420","2121","10305","63769","51931"])]

,itemID,title,author,publisher,main topic,subtopics
490,51931,Der Gebannte,Alexey Pehov,Piper Verlag GmbH,FMB,[]
855,63769,Dunkler Herrscher,Marc Stehle,Eisermann,FMB,"[FMT,FMW]"
1561,10305,Die Greifen-Saga - Die komplette Trilogie im S...,C. M. Spoerri,Sternensand Verlag,FMB,[FN]
5074,44420,Damaris: Das Vermächtnis der Wüstenzwerge,C. M. Spoerri,Sternensand Verlag,FMB,[FN]
5270,2121,Damaris (Band 1): Der Greifenorden von Chakas,C. M. Spoerri,Sternensand Verlag,FMB,[]
5808,56794,Damaris (Band 2): Der Ring des Fürsten,C. M. Spoerri,Sternensand Verlag,FMB,[FN]


In [ ]:
for item in recommendations.keys():
    recommendations[]